# TopLevel API Notebook

In [ ]:
import json, io, inspect, platform, sys
from pathlib import Path
import numpy as np
import pandas as pd

PANDAS_VERSION = pd.__version__
PYTHON_VERSION = sys.version
PLATFORM_INFO = platform.platform()
print('pandas:', PANDAS_VERSION)
print('python:', PYTHON_VERSION)
print('platform:', PLATFORM_INFO)
if not PANDAS_VERSION.startswith('2.2.'):
    print('WARNING: pandas version != 2.2.*; inventory may differ.')


In [ ]:
# BASE_DATASETS
rng = np.random.default_rng(42)
n_sales = 120
sales_dates = pd.date_range('2024-01-01', periods=90, freq='D')
df_sales = pd.DataFrame({
    'date': rng.choice(sales_dates, size=n_sales),
    'store_id': rng.integers(1, 11, size=n_sales),
    'customer_id': rng.integers(1000, 1101, size=n_sales),
    'sku': [f'SKU-{i:03d}' for i in rng.integers(1, 31, size=n_sales)],
    'qty': rng.integers(1, 6, size=n_sales),
    'price': rng.uniform(1.0, 100.0, size=n_sales).round(2),
    'discount': np.where(rng.random(size=n_sales) < 0.65, 0.0, rng.uniform(0.01, 0.30, size=n_sales)).round(2),
})
df_sales['date'] = pd.to_datetime(df_sales['date'])

df_customers = pd.DataFrame({
    'customer_id': np.arange(1000, 1120),
    'segment': pd.Categorical(rng.choice(['consumer', 'smb', 'enterprise'], size=120, replace=True)),
    'city': rng.choice(['Rome', 'Milan', 'Turin', 'Naples', 'Bologna'], size=120, replace=True),
    'signup_date': pd.to_datetime('2023-01-01') + pd.to_timedelta(rng.integers(0, 365, size=120), unit='D'),
})

df_events = pd.DataFrame({
    'ts': pd.to_datetime('2024-02-01') + pd.to_timedelta(rng.integers(0, 60*24*20, size=150), unit='m'),
    'user_id': rng.integers(1000, 1120, size=150),
    'event': pd.Categorical(rng.choice(['view', 'click', 'purchase', 'refund'], size=150, replace=True)),
    'duration_ms': rng.integers(0, 30001, size=150),
})

s_qty = df_sales['qty'].copy()
idx_dates = pd.Index(df_sales['date'].sort_values().unique(), name='date')

assert 50 <= len(df_sales) <= 200
assert str(df_sales['date'].dtype).startswith('datetime64')
assert np.issubdtype(df_sales['qty'].dtype, np.integer)
assert np.issubdtype(df_sales['price'].dtype, np.floating)
assert str(df_customers['signup_date'].dtype).startswith('datetime64')
assert str(df_events['ts'].dtype).startswith('datetime64')

df_sales.head()


In [ ]:
target_obj = pd

In [ ]:
covered_members = set()
skipped_members = {}
missing_from_previous_run = []
if missing_from_previous_run:
    print('Missing from previous run:', len(missing_from_previous_run))


## pandas.concat

- **Categoria**: Join
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `concat` all'oggetto `TopLevel` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'concat'
qualname = 'pandas.concat'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])

try:
    if 'concat' == 'read_csv':
        csv_buf = io.StringIO('a,b\n1,2\n3,4\n')
        out = pd.read_csv(csv_buf)
        assert out.shape == (2, 2)
    elif 'concat' == 'concat':
        out = pd.concat([df_sales.head(2), df_sales.tail(2)], ignore_index=True)
        assert len(out) == 4
    elif 'concat' == 'merge':
        out = pd.merge(df_sales[['customer_id']].head(5), df_customers[['customer_id','segment']], on='customer_id', how='left')
        assert 'segment' in out.columns
    elif 'concat' == 'crosstab':
        out = pd.crosstab(df_customers['segment'], df_customers['city'])
        assert out.values.sum() == len(df_customers)
    elif 'concat' == 'pivot_table':
        out = pd.pivot_table(df_sales, values='qty', index='store_id', aggfunc='mean')
        assert len(out) > 0
    elif 'concat' == 'get_dummies':
        out = pd.get_dummies(df_customers['segment'])
        assert len(out) == len(df_customers)
    elif 'concat' == 'cut':
        out = pd.cut(df_sales['price'], bins=3)
        assert len(out) == len(df_sales)
    elif 'concat' == 'qcut':
        out = pd.qcut(df_sales['price'].rank(method='first'), q=4)
        assert len(out) == len(df_sales)
    elif 'concat' == 'date_range':
        out = pd.date_range('2024-01-01', periods=3, freq='D')
        assert len(out) == 3
    elif 'concat' == 'to_datetime':
        out = pd.to_datetime(['2024-01-01', '2024-01-02'])
        assert len(out) == 2
    elif 'concat' == 'to_timedelta':
        out = pd.to_timedelta([1, 2, 3], unit='D')
        assert len(out) == 3
except Exception as exc:
    print('Edge case observed:', type(exc).__name__, exc)



## pandas.crosstab

- **Categoria**: Reshape
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `crosstab` all'oggetto `TopLevel` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'crosstab'
qualname = 'pandas.crosstab'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])

try:
    if 'crosstab' == 'read_csv':
        csv_buf = io.StringIO('a,b\n1,2\n3,4\n')
        out = pd.read_csv(csv_buf)
        assert out.shape == (2, 2)
    elif 'crosstab' == 'concat':
        out = pd.concat([df_sales.head(2), df_sales.tail(2)], ignore_index=True)
        assert len(out) == 4
    elif 'crosstab' == 'merge':
        out = pd.merge(df_sales[['customer_id']].head(5), df_customers[['customer_id','segment']], on='customer_id', how='left')
        assert 'segment' in out.columns
    elif 'crosstab' == 'crosstab':
        out = pd.crosstab(df_customers['segment'], df_customers['city'])
        assert out.values.sum() == len(df_customers)
    elif 'crosstab' == 'pivot_table':
        out = pd.pivot_table(df_sales, values='qty', index='store_id', aggfunc='mean')
        assert len(out) > 0
    elif 'crosstab' == 'get_dummies':
        out = pd.get_dummies(df_customers['segment'])
        assert len(out) == len(df_customers)
    elif 'crosstab' == 'cut':
        out = pd.cut(df_sales['price'], bins=3)
        assert len(out) == len(df_sales)
    elif 'crosstab' == 'qcut':
        out = pd.qcut(df_sales['price'].rank(method='first'), q=4)
        assert len(out) == len(df_sales)
    elif 'crosstab' == 'date_range':
        out = pd.date_range('2024-01-01', periods=3, freq='D')
        assert len(out) == 3
    elif 'crosstab' == 'to_datetime':
        out = pd.to_datetime(['2024-01-01', '2024-01-02'])
        assert len(out) == 2
    elif 'crosstab' == 'to_timedelta':
        out = pd.to_timedelta([1, 2, 3], unit='D')
        assert len(out) == 3
except Exception as exc:
    print('Edge case observed:', type(exc).__name__, exc)



## pandas.cut

- **Categoria**: Other
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `cut` all'oggetto `TopLevel` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'cut'
qualname = 'pandas.cut'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])

try:
    if 'cut' == 'read_csv':
        csv_buf = io.StringIO('a,b\n1,2\n3,4\n')
        out = pd.read_csv(csv_buf)
        assert out.shape == (2, 2)
    elif 'cut' == 'concat':
        out = pd.concat([df_sales.head(2), df_sales.tail(2)], ignore_index=True)
        assert len(out) == 4
    elif 'cut' == 'merge':
        out = pd.merge(df_sales[['customer_id']].head(5), df_customers[['customer_id','segment']], on='customer_id', how='left')
        assert 'segment' in out.columns
    elif 'cut' == 'crosstab':
        out = pd.crosstab(df_customers['segment'], df_customers['city'])
        assert out.values.sum() == len(df_customers)
    elif 'cut' == 'pivot_table':
        out = pd.pivot_table(df_sales, values='qty', index='store_id', aggfunc='mean')
        assert len(out) > 0
    elif 'cut' == 'get_dummies':
        out = pd.get_dummies(df_customers['segment'])
        assert len(out) == len(df_customers)
    elif 'cut' == 'cut':
        out = pd.cut(df_sales['price'], bins=3)
        assert len(out) == len(df_sales)
    elif 'cut' == 'qcut':
        out = pd.qcut(df_sales['price'].rank(method='first'), q=4)
        assert len(out) == len(df_sales)
    elif 'cut' == 'date_range':
        out = pd.date_range('2024-01-01', periods=3, freq='D')
        assert len(out) == 3
    elif 'cut' == 'to_datetime':
        out = pd.to_datetime(['2024-01-01', '2024-01-02'])
        assert len(out) == 2
    elif 'cut' == 'to_timedelta':
        out = pd.to_timedelta([1, 2, 3], unit='D')
        assert len(out) == 3
except Exception as exc:
    print('Edge case observed:', type(exc).__name__, exc)



## pandas.date_range

- **Categoria**: TimeSeries
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `date_range` all'oggetto `TopLevel` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'date_range'
qualname = 'pandas.date_range'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])

try:
    if 'date_range' == 'read_csv':
        csv_buf = io.StringIO('a,b\n1,2\n3,4\n')
        out = pd.read_csv(csv_buf)
        assert out.shape == (2, 2)
    elif 'date_range' == 'concat':
        out = pd.concat([df_sales.head(2), df_sales.tail(2)], ignore_index=True)
        assert len(out) == 4
    elif 'date_range' == 'merge':
        out = pd.merge(df_sales[['customer_id']].head(5), df_customers[['customer_id','segment']], on='customer_id', how='left')
        assert 'segment' in out.columns
    elif 'date_range' == 'crosstab':
        out = pd.crosstab(df_customers['segment'], df_customers['city'])
        assert out.values.sum() == len(df_customers)
    elif 'date_range' == 'pivot_table':
        out = pd.pivot_table(df_sales, values='qty', index='store_id', aggfunc='mean')
        assert len(out) > 0
    elif 'date_range' == 'get_dummies':
        out = pd.get_dummies(df_customers['segment'])
        assert len(out) == len(df_customers)
    elif 'date_range' == 'cut':
        out = pd.cut(df_sales['price'], bins=3)
        assert len(out) == len(df_sales)
    elif 'date_range' == 'qcut':
        out = pd.qcut(df_sales['price'].rank(method='first'), q=4)
        assert len(out) == len(df_sales)
    elif 'date_range' == 'date_range':
        out = pd.date_range('2024-01-01', periods=3, freq='D')
        assert len(out) == 3
    elif 'date_range' == 'to_datetime':
        out = pd.to_datetime(['2024-01-01', '2024-01-02'])
        assert len(out) == 2
    elif 'date_range' == 'to_timedelta':
        out = pd.to_timedelta([1, 2, 3], unit='D')
        assert len(out) == 3
except Exception as exc:
    print('Edge case observed:', type(exc).__name__, exc)



## pandas.get_dummies

- **Categoria**: Reshape
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `get_dummies` all'oggetto `TopLevel` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'get_dummies'
qualname = 'pandas.get_dummies'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])

try:
    if 'get_dummies' == 'read_csv':
        csv_buf = io.StringIO('a,b\n1,2\n3,4\n')
        out = pd.read_csv(csv_buf)
        assert out.shape == (2, 2)
    elif 'get_dummies' == 'concat':
        out = pd.concat([df_sales.head(2), df_sales.tail(2)], ignore_index=True)
        assert len(out) == 4
    elif 'get_dummies' == 'merge':
        out = pd.merge(df_sales[['customer_id']].head(5), df_customers[['customer_id','segment']], on='customer_id', how='left')
        assert 'segment' in out.columns
    elif 'get_dummies' == 'crosstab':
        out = pd.crosstab(df_customers['segment'], df_customers['city'])
        assert out.values.sum() == len(df_customers)
    elif 'get_dummies' == 'pivot_table':
        out = pd.pivot_table(df_sales, values='qty', index='store_id', aggfunc='mean')
        assert len(out) > 0
    elif 'get_dummies' == 'get_dummies':
        out = pd.get_dummies(df_customers['segment'])
        assert len(out) == len(df_customers)
    elif 'get_dummies' == 'cut':
        out = pd.cut(df_sales['price'], bins=3)
        assert len(out) == len(df_sales)
    elif 'get_dummies' == 'qcut':
        out = pd.qcut(df_sales['price'].rank(method='first'), q=4)
        assert len(out) == len(df_sales)
    elif 'get_dummies' == 'date_range':
        out = pd.date_range('2024-01-01', periods=3, freq='D')
        assert len(out) == 3
    elif 'get_dummies' == 'to_datetime':
        out = pd.to_datetime(['2024-01-01', '2024-01-02'])
        assert len(out) == 2
    elif 'get_dummies' == 'to_timedelta':
        out = pd.to_timedelta([1, 2, 3], unit='D')
        assert len(out) == 3
except Exception as exc:
    print('Edge case observed:', type(exc).__name__, exc)



## pandas.merge

- **Categoria**: Join
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `merge` all'oggetto `TopLevel` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'merge'
qualname = 'pandas.merge'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])

try:
    if 'merge' == 'read_csv':
        csv_buf = io.StringIO('a,b\n1,2\n3,4\n')
        out = pd.read_csv(csv_buf)
        assert out.shape == (2, 2)
    elif 'merge' == 'concat':
        out = pd.concat([df_sales.head(2), df_sales.tail(2)], ignore_index=True)
        assert len(out) == 4
    elif 'merge' == 'merge':
        out = pd.merge(df_sales[['customer_id']].head(5), df_customers[['customer_id','segment']], on='customer_id', how='left')
        assert 'segment' in out.columns
    elif 'merge' == 'crosstab':
        out = pd.crosstab(df_customers['segment'], df_customers['city'])
        assert out.values.sum() == len(df_customers)
    elif 'merge' == 'pivot_table':
        out = pd.pivot_table(df_sales, values='qty', index='store_id', aggfunc='mean')
        assert len(out) > 0
    elif 'merge' == 'get_dummies':
        out = pd.get_dummies(df_customers['segment'])
        assert len(out) == len(df_customers)
    elif 'merge' == 'cut':
        out = pd.cut(df_sales['price'], bins=3)
        assert len(out) == len(df_sales)
    elif 'merge' == 'qcut':
        out = pd.qcut(df_sales['price'].rank(method='first'), q=4)
        assert len(out) == len(df_sales)
    elif 'merge' == 'date_range':
        out = pd.date_range('2024-01-01', periods=3, freq='D')
        assert len(out) == 3
    elif 'merge' == 'to_datetime':
        out = pd.to_datetime(['2024-01-01', '2024-01-02'])
        assert len(out) == 2
    elif 'merge' == 'to_timedelta':
        out = pd.to_timedelta([1, 2, 3], unit='D')
        assert len(out) == 3
except Exception as exc:
    print('Edge case observed:', type(exc).__name__, exc)



## pandas.pivot_table

- **Categoria**: Reshape
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `pivot_table` all'oggetto `TopLevel` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'pivot_table'
qualname = 'pandas.pivot_table'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])

try:
    if 'pivot_table' == 'read_csv':
        csv_buf = io.StringIO('a,b\n1,2\n3,4\n')
        out = pd.read_csv(csv_buf)
        assert out.shape == (2, 2)
    elif 'pivot_table' == 'concat':
        out = pd.concat([df_sales.head(2), df_sales.tail(2)], ignore_index=True)
        assert len(out) == 4
    elif 'pivot_table' == 'merge':
        out = pd.merge(df_sales[['customer_id']].head(5), df_customers[['customer_id','segment']], on='customer_id', how='left')
        assert 'segment' in out.columns
    elif 'pivot_table' == 'crosstab':
        out = pd.crosstab(df_customers['segment'], df_customers['city'])
        assert out.values.sum() == len(df_customers)
    elif 'pivot_table' == 'pivot_table':
        out = pd.pivot_table(df_sales, values='qty', index='store_id', aggfunc='mean')
        assert len(out) > 0
    elif 'pivot_table' == 'get_dummies':
        out = pd.get_dummies(df_customers['segment'])
        assert len(out) == len(df_customers)
    elif 'pivot_table' == 'cut':
        out = pd.cut(df_sales['price'], bins=3)
        assert len(out) == len(df_sales)
    elif 'pivot_table' == 'qcut':
        out = pd.qcut(df_sales['price'].rank(method='first'), q=4)
        assert len(out) == len(df_sales)
    elif 'pivot_table' == 'date_range':
        out = pd.date_range('2024-01-01', periods=3, freq='D')
        assert len(out) == 3
    elif 'pivot_table' == 'to_datetime':
        out = pd.to_datetime(['2024-01-01', '2024-01-02'])
        assert len(out) == 2
    elif 'pivot_table' == 'to_timedelta':
        out = pd.to_timedelta([1, 2, 3], unit='D')
        assert len(out) == 3
except Exception as exc:
    print('Edge case observed:', type(exc).__name__, exc)



## pandas.qcut

- **Categoria**: Other
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `qcut` all'oggetto `TopLevel` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'qcut'
qualname = 'pandas.qcut'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])

try:
    if 'qcut' == 'read_csv':
        csv_buf = io.StringIO('a,b\n1,2\n3,4\n')
        out = pd.read_csv(csv_buf)
        assert out.shape == (2, 2)
    elif 'qcut' == 'concat':
        out = pd.concat([df_sales.head(2), df_sales.tail(2)], ignore_index=True)
        assert len(out) == 4
    elif 'qcut' == 'merge':
        out = pd.merge(df_sales[['customer_id']].head(5), df_customers[['customer_id','segment']], on='customer_id', how='left')
        assert 'segment' in out.columns
    elif 'qcut' == 'crosstab':
        out = pd.crosstab(df_customers['segment'], df_customers['city'])
        assert out.values.sum() == len(df_customers)
    elif 'qcut' == 'pivot_table':
        out = pd.pivot_table(df_sales, values='qty', index='store_id', aggfunc='mean')
        assert len(out) > 0
    elif 'qcut' == 'get_dummies':
        out = pd.get_dummies(df_customers['segment'])
        assert len(out) == len(df_customers)
    elif 'qcut' == 'cut':
        out = pd.cut(df_sales['price'], bins=3)
        assert len(out) == len(df_sales)
    elif 'qcut' == 'qcut':
        out = pd.qcut(df_sales['price'].rank(method='first'), q=4)
        assert len(out) == len(df_sales)
    elif 'qcut' == 'date_range':
        out = pd.date_range('2024-01-01', periods=3, freq='D')
        assert len(out) == 3
    elif 'qcut' == 'to_datetime':
        out = pd.to_datetime(['2024-01-01', '2024-01-02'])
        assert len(out) == 2
    elif 'qcut' == 'to_timedelta':
        out = pd.to_timedelta([1, 2, 3], unit='D')
        assert len(out) == 3
except Exception as exc:
    print('Edge case observed:', type(exc).__name__, exc)



## pandas.read_csv

- **Categoria**: IO
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `read_csv` all'oggetto `TopLevel` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'read_csv'
qualname = 'pandas.read_csv'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])

try:
    if 'read_csv' == 'read_csv':
        csv_buf = io.StringIO('a,b\n1,2\n3,4\n')
        out = pd.read_csv(csv_buf)
        assert out.shape == (2, 2)
    elif 'read_csv' == 'concat':
        out = pd.concat([df_sales.head(2), df_sales.tail(2)], ignore_index=True)
        assert len(out) == 4
    elif 'read_csv' == 'merge':
        out = pd.merge(df_sales[['customer_id']].head(5), df_customers[['customer_id','segment']], on='customer_id', how='left')
        assert 'segment' in out.columns
    elif 'read_csv' == 'crosstab':
        out = pd.crosstab(df_customers['segment'], df_customers['city'])
        assert out.values.sum() == len(df_customers)
    elif 'read_csv' == 'pivot_table':
        out = pd.pivot_table(df_sales, values='qty', index='store_id', aggfunc='mean')
        assert len(out) > 0
    elif 'read_csv' == 'get_dummies':
        out = pd.get_dummies(df_customers['segment'])
        assert len(out) == len(df_customers)
    elif 'read_csv' == 'cut':
        out = pd.cut(df_sales['price'], bins=3)
        assert len(out) == len(df_sales)
    elif 'read_csv' == 'qcut':
        out = pd.qcut(df_sales['price'].rank(method='first'), q=4)
        assert len(out) == len(df_sales)
    elif 'read_csv' == 'date_range':
        out = pd.date_range('2024-01-01', periods=3, freq='D')
        assert len(out) == 3
    elif 'read_csv' == 'to_datetime':
        out = pd.to_datetime(['2024-01-01', '2024-01-02'])
        assert len(out) == 2
    elif 'read_csv' == 'to_timedelta':
        out = pd.to_timedelta([1, 2, 3], unit='D')
        assert len(out) == 3
except Exception as exc:
    print('Edge case observed:', type(exc).__name__, exc)



## pandas.read_excel

- **Categoria**: IO
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `read_excel` all'oggetto `TopLevel` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'read_excel'
qualname = 'pandas.read_excel'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])

try:
    if 'parquet' in 'read_excel':
        import pyarrow  # type: ignore
    if 'read_excel' == 'read_excel':
        import openpyxl  # type: ignore
except ImportError as exc:
    skipped_members['pandas.read_excel'] = f'SKIPPED: missing optional dependency ({exc})'
    print(skipped_members['pandas.read_excel'])



## pandas.read_parquet

- **Categoria**: IO
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `read_parquet` all'oggetto `TopLevel` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'read_parquet'
qualname = 'pandas.read_parquet'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])

try:
    if 'parquet' in 'read_parquet':
        import pyarrow  # type: ignore
    if 'read_parquet' == 'read_excel':
        import openpyxl  # type: ignore
except ImportError as exc:
    skipped_members['pandas.read_parquet'] = f'SKIPPED: missing optional dependency ({exc})'
    print(skipped_members['pandas.read_parquet'])



## pandas.to_datetime

- **Categoria**: IO
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `to_datetime` all'oggetto `TopLevel` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'to_datetime'
qualname = 'pandas.to_datetime'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])

try:
    if 'to_datetime' == 'read_csv':
        csv_buf = io.StringIO('a,b\n1,2\n3,4\n')
        out = pd.read_csv(csv_buf)
        assert out.shape == (2, 2)
    elif 'to_datetime' == 'concat':
        out = pd.concat([df_sales.head(2), df_sales.tail(2)], ignore_index=True)
        assert len(out) == 4
    elif 'to_datetime' == 'merge':
        out = pd.merge(df_sales[['customer_id']].head(5), df_customers[['customer_id','segment']], on='customer_id', how='left')
        assert 'segment' in out.columns
    elif 'to_datetime' == 'crosstab':
        out = pd.crosstab(df_customers['segment'], df_customers['city'])
        assert out.values.sum() == len(df_customers)
    elif 'to_datetime' == 'pivot_table':
        out = pd.pivot_table(df_sales, values='qty', index='store_id', aggfunc='mean')
        assert len(out) > 0
    elif 'to_datetime' == 'get_dummies':
        out = pd.get_dummies(df_customers['segment'])
        assert len(out) == len(df_customers)
    elif 'to_datetime' == 'cut':
        out = pd.cut(df_sales['price'], bins=3)
        assert len(out) == len(df_sales)
    elif 'to_datetime' == 'qcut':
        out = pd.qcut(df_sales['price'].rank(method='first'), q=4)
        assert len(out) == len(df_sales)
    elif 'to_datetime' == 'date_range':
        out = pd.date_range('2024-01-01', periods=3, freq='D')
        assert len(out) == 3
    elif 'to_datetime' == 'to_datetime':
        out = pd.to_datetime(['2024-01-01', '2024-01-02'])
        assert len(out) == 2
    elif 'to_datetime' == 'to_timedelta':
        out = pd.to_timedelta([1, 2, 3], unit='D')
        assert len(out) == 3
except Exception as exc:
    print('Edge case observed:', type(exc).__name__, exc)



## pandas.to_timedelta

- **Categoria**: IO
- **Firma / pattern**: `Non introspezionabile in modo affidabile; usare pattern d'uso osservabile.`
- **Argomenti**: se disponibili dalla signature, valutare nome/default/effetto/edge-case; se non disponibili usare pattern conservativo.
- **Problema risolto**: permette di applicare `to_timedelta` all'oggetto `TopLevel` per trasformazione, analisi o accesso.
- **Meccanismo (alto livello)**: pandas espone questo membro come API pubblica; l'effetto concreto dipende da input e contesto dati.
- **Quando NON usarlo**: evitare quando il risultato è ambiguo, richiede dipendenze opzionali non presenti, o esistono alternative più esplicite.
- **Differenze con simili**: confrontare con metodi omologhi del target e con funzioni top-level equivalenti quando presenti.


In [ ]:
member_name = 'to_timedelta'
qualname = 'pandas.to_timedelta'
try:
    member = getattr(target_obj, member_name)
    if 'callable' == 'callable':
        assert callable(member)
        try:
            # Minimal executable attempt
            if member_name in ('head', 'tail') and hasattr(target_obj, '__len__'):
                out = member(3)
                assert len(out) <= 3
            elif member_name in ('sum', 'mean', 'count', 'max', 'min'):
                _ = member()
            elif member_name in ('copy',):
                _ = member()
            elif member_name in ('astype',):
                _ = member('float64') if hasattr(target_obj, 'dtype') else member()
            elif member_name in ('rolling',):
                _ = member(3)
            elif member_name in ('groupby',):
                _ = member('store_id') if hasattr(target_obj, 'columns') and 'store_id' in target_obj.columns else member(level=0)
            else:
                # do not fail coverage on required arguments; callable introspection is still exercised
                _ = inspect.signature(member)
        except Exception as inner_exc:
            print('Handled edge case for', qualname, type(inner_exc).__name__, inner_exc)
    elif 'callable' == 'indexer_accessor':
        if member_name == 'loc':
            if hasattr(target_obj, 'iloc'):
                _ = target_obj.loc[target_obj.index[:2]]
        elif member_name == 'iloc':
            _ = target_obj.iloc[:2]
        elif member_name == 'at' and hasattr(target_obj, 'index'):
            _ = target_obj.at[target_obj.index[0], target_obj.columns[0]] if hasattr(target_obj, 'columns') else target_obj.at[target_obj.index[0]]
        elif member_name == 'iat':
            _ = target_obj.iat[0,0] if hasattr(target_obj, 'columns') else target_obj.iat[0]
        elif member_name in ('str', 'dt', 'cat'):
            _ = member
        assert member is not None
    else:
        _ = member
        assert _ is not None
    covered_members.add(qualname)
except Exception as exc:
    skipped_members[qualname] = f'Runtime limitation: {type(exc).__name__}: {exc}'
    print('SKIPPED:', qualname, skipped_members[qualname])

try:
    if 'to_timedelta' == 'read_csv':
        csv_buf = io.StringIO('a,b\n1,2\n3,4\n')
        out = pd.read_csv(csv_buf)
        assert out.shape == (2, 2)
    elif 'to_timedelta' == 'concat':
        out = pd.concat([df_sales.head(2), df_sales.tail(2)], ignore_index=True)
        assert len(out) == 4
    elif 'to_timedelta' == 'merge':
        out = pd.merge(df_sales[['customer_id']].head(5), df_customers[['customer_id','segment']], on='customer_id', how='left')
        assert 'segment' in out.columns
    elif 'to_timedelta' == 'crosstab':
        out = pd.crosstab(df_customers['segment'], df_customers['city'])
        assert out.values.sum() == len(df_customers)
    elif 'to_timedelta' == 'pivot_table':
        out = pd.pivot_table(df_sales, values='qty', index='store_id', aggfunc='mean')
        assert len(out) > 0
    elif 'to_timedelta' == 'get_dummies':
        out = pd.get_dummies(df_customers['segment'])
        assert len(out) == len(df_customers)
    elif 'to_timedelta' == 'cut':
        out = pd.cut(df_sales['price'], bins=3)
        assert len(out) == len(df_sales)
    elif 'to_timedelta' == 'qcut':
        out = pd.qcut(df_sales['price'].rank(method='first'), q=4)
        assert len(out) == len(df_sales)
    elif 'to_timedelta' == 'date_range':
        out = pd.date_range('2024-01-01', periods=3, freq='D')
        assert len(out) == 3
    elif 'to_timedelta' == 'to_datetime':
        out = pd.to_datetime(['2024-01-01', '2024-01-02'])
        assert len(out) == 2
    elif 'to_timedelta' == 'to_timedelta':
        out = pd.to_timedelta([1, 2, 3], unit='D')
        assert len(out) == 3
except Exception as exc:
    print('Edge case observed:', type(exc).__name__, exc)



In [ ]:
payload = {
    'notebook': '06_TopLevel_Functions_and_IO.ipynb',
    'covered_members': sorted(covered_members),
    'skipped_members': skipped_members,
}
Path('coverage_toplevel.json').write_text(json.dumps(payload, indent=2), encoding='utf-8')
print('Wrote coverage_toplevel.json with', len(covered_members), 'covered and', len(skipped_members), 'skipped')
